In [34]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

In [38]:
df = pd.read_csv('com.csv')
carbon_number = df['component']
molar_fraction = df['molar_percentage']
molecular_weight = df['molar_mass']
density = df['density']

carbonnumber_max = carbon_number.values[-2]
molar_fraction_max = molar_fraction.values[-2]
molecular_weight_max = molecular_weight.values[-2]

Primero ajusto los parámetros para obtener todas las constantes intervinientes

In [39]:
from recore import Foo_fit
foo_fit = Foo_fit()

A,B = foo_fit.fit_AB(carbon_number,molar_fraction)
L,M = foo_fit.fit_LM(carbon_number, density)
Ac, Bc = foo_fit.fit_AcBc(carbon_number, molar_fraction)
Ad = foo_fit.fit_Ad(carbon_number, density)[0]
C = foo_fit.fit_C(carbon_number,molecular_weight) [0]

fit_parameters = {
    'A':A, 'B':B, 'L':L, 'M':M, 'Ac':Ac, 'Bc':Bc
    ,'Ad':Ad,'C':C
}

fit_parameters

{'A': 5.269814253877877,
 'B': -2.693453959800989,
 'L': 0.5181480379693734,
 'M': 0.2693755951739469,
 'Ac': 2.9969050222015183,
 'Bc': -87.60869574376241,
 'Ad': -0.4578051749233114,
 'C': 13.913346938986416}

Para no utilizar los valores establecidos aleatoriamente por Pedersen para el número máximo de carbono
calculo el valor máximo con el criterio de Cismondi et. al

In [41]:
from recore import Residual_fraction

residual_fraction = Residual_fraction(molecular_weight_max,molar_fraction_max)

carbon_range_it = list(range(carbonnumber_max,201,1))
res_carbon_number_mx = residual_fraction.carbon_number_max(carbon_range_it, Ac, Bc, C)

IndexError: invalid index to scalar variable.